In [13]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

import os
import cv2
import numpy as np

# === Correct import from model.py ===
from model.model import DecomNet, RelightNet

def enhance_image_retinexnet(input_path, output_path, checkpoint_dir='checkpoint/'):
    # Load input image
    input_img = cv2.imread(input_path)
    input_img = cv2.cvtColor(input_img, cv2.COLOR_BGR2RGB)
    input_img = input_img.astype(np.float32) / 255.0
    input_img = np.expand_dims(input_img, axis=0)

    sess = tf.Session()

    input_tensor = tf.placeholder(tf.float32, [None, None, None, 3], name='input_low')

    # === Build network
    decom_net = DecomNet()
    relight_net = RelightNet()

    R_low, I_low = decom_net.build(input_tensor)
    I_enhanced = relight_net.build(I_low)

    var_Decom = [var for var in tf.global_variables() if 'DecomNet' in var.name]
    var_Relight = [var for var in tf.global_variables() if 'RelightNet' in var.name]

    saver_Decom = tf.train.Saver(var_list=var_Decom)
    saver_Relight = tf.train.Saver(var_list=var_Relight)

    sess.run(tf.global_variables_initializer())

    print('Restoring DecomNet...')
    saver_Decom.restore(sess, os.path.join(checkpoint_dir, 'RetinexNet-tensorflow'))
    print('Restoring RelightNet...')
    saver_Relight.restore(sess, os.path.join(checkpoint_dir, 'RetinexNet-tensorflow'))

    feed_dict = {input_tensor: input_img}
    output_R_low, output_I_low, output_I_enhanced = sess.run([R_low, I_low, I_enhanced], feed_dict=feed_dict)

    output_S = output_R_low * output_I_enhanced
    output_S = np.squeeze(output_S)
    output_S = np.clip(output_S * 255.0, 0, 255).astype(np.uint8)
    output_S = cv2.cvtColor(output_S, cv2.COLOR_RGB2BGR)

    if not os.path.exists(os.path.dirname(output_path)):
        os.makedirs(os.path.dirname(output_path))
    cv2.imwrite(output_path, output_S)

    print(f'✅ Enhanced image saved at: {output_path}')
    sess.close()

# Example
if __name__ == "__main__":
    input_image_path = 'ADD6379.jpg'
    output_image_path = 'ADD6379_enhanced.jpg'
    enhance_image_retinexnet(input_image_path, output_image_path)


TypeError: DecomNet() missing 2 required positional arguments: 'input_im' and 'layer_num'